In [1]:
print('hello, final work')

hello, final work


In [5]:
import pandas as pd

df = pd.read_csv("jokes.csv")

# Удалить дубликаты
df = df.drop_duplicates()

# Удалить строки с пустыми значениями
df = df.dropna()

df.to_csv("jokes.csv")

In [21]:
import csv

with open('jokes.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    jokes = list(reader)


with open('jokes_test.csv', 'w', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerows(jokes[:100])

In [12]:
df.head(10)

,Unnamed: 0,theme,text,rating
0,0,pro-sudey,На суде в Стамбуле обвиняемый сказал:\r\n- На...,5
1,1,pro-sudey,"- Вы продолжаете утверждать, что обвиняемый н...",4
2,2,pro-sudey,"На суде.\r\n- Итак, когда дело дошло до столкн...",0
3,3,pro-sudey,Старую леди сбил автомобиль. На суде ее спраши...,4
4,4,pro-sudey,"Судья говорит:\r\n- Согласно вашей жалобе, об...",2
5,5,pro-sudey,На судебном заседании.\r\n- Гражданка Дроздова...,3
6,6,pro-sudey,- Драка происходила так. Одной рукой я схват...,-3
7,7,pro-sudey,"Судья:\r\n- Свидетель, вы должны говорить прав...",2
8,8,pro-sudey,"Судья подсудимому:\r\n- Ну-ну, перестаньте вол...",-2
9,9,pro-sudey,"Судья спрашивает четырех индейцев, убежавших ...",4


In [14]:
!pip install torch

     -------------------------------------- 192.3/192.3 MB 4.4 MB/s eta 0:00:00
  Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
  Using cached networkx-3.1-py3-none-any.whl (2.1 MB)
  Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)



[notice] A new release of pip available: 22.3.1 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# pip install torch -f https://download.pytorch.org/whl/cpu/torch_stable.html

In [34]:
import pandas as pd
from transformers import GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments, GPT2LMHeadModel
import os

# Определение токенизатора
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")
tokenizer.pad_token = tokenizer.eos_token

def read_jokes_csv(filename):
    df = pd.read_csv(filename, usecols=['text'])
    return df

def train_and_save_model():
    # Загрузка данных
    df = read_jokes_csv("jokes.csv")
    df = df.head(20)
    train_df = df.sample(frac=0.8, random_state=42)
    eval_df = df.drop(train_df.index)

    # Сохранение данных в файлы
    train_path = "jokes_train.txt"
    test_path = "jokes_test.txt"
    train_df['text'].to_csv(train_path, index=False, header=False)
    eval_df['text'].to_csv(test_path, index=False, header=False)

    # Токенизация данных
    train_dataset = TextDataset(tokenizer=tokenizer, file_path=train_path, block_size=128, overwrite_cache=True)
    test_dataset = TextDataset(tokenizer=tokenizer, file_path=test_path, block_size=128, overwrite_cache=True)
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    # Определение параметров обучения
    training_args = TrainingArguments(
        output_dir="./jokes_model",
        overwrite_output_dir=True,
        num_train_epochs=2,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        logging_dir="./logs",
        logging_steps=10,
        save_steps=10,
        eval_steps=10,
        save_total_limit=2,
        evaluation_strategy="steps"
    )

    # Обучение модели
    model = GPT2LMHeadModel.from_pretrained("gpt2-medium")
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
    )

    # Тренировка модели
    training_results = trainer.train()
    trainer.save_model("./jokes_model")

    # Возвращаем среднее значение потерь для всего обучения
    return model, tokenizer, training_results.training_loss

def load_and_test_model(model_path, test_path):
    # Загрузка модели и токенизатора из файла
    model = GPT2LMHeadModel.from_pretrained(model_path)
    tokenizer = GPT2Tokenizer.from_pretrained(model_path)
    tokenizer.pad_token = tokenizer.eos_token

    # Токенизация тестовых данных
    test_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=test_path,
        block_size=128,
        overwrite_cache=True
    )

    # Оценка модели на тестовых данных
    training_args = TrainingArguments(
        output_dir="./results_test",
        per_device_eval_batch_size=8,
        logging_dir="./logs_test",
        logging_steps=10,
        eval_steps=10,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=DataCollatorForLanguageModeling(
            tokenizer=tokenizer, mlm=False
        ),
        eval_dataset=test_dataset,
    )

    results = trainer.evaluate()

    print("Evaluation Results:", results)

    # Генерация текста на основе обученной модели
    input_text = "На суде"
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    # output = model.generate(input_ids, max_length=100, num_return_sequences=5, temperature=0.7)
    output = model.generate(input_ids, max_length=100, num_return_sequences=5, num_beams=5, temperature=0.7)
    for i, generated_text in enumerate(tokenizer.decode(o, skip_special_tokens=True) for o in output):
        print(f"Generated text {i + 1}:")
        print(generated_text)



if __name__ == "__main__":
    # Обучение и сохранение модели
    model, tokenizer, average_loss = train_and_save_model()

    # Вывод числового значения среднего loss на обучении
    print(f"Avg Training Loss: {average_loss}")

    # Загрузка модели и оценка на тестовых данных
    load_and_test_model("./jokes_model", "jokes_test.txt")

D:\data\innopolis\itog\venv\Lib\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss,Validation Loss


Avg Training Loss: 2.429222742716471


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
D:\data\innopolis\itog\venv\Lib\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


D:\data\innopolis\itog\venv\Lib\site-packages\transformers\generation\configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Evaluation Results: {'eval_loss': 1.902648687362671, 'eval_runtime': 2.527, 'eval_samples_per_second': 1.979, 'eval_steps_per_second': 0.396}
Generated text 1:
На судельствуются вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы
Generated text 2:
На судельствуются, вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы в
Generated text 3:
На судельствуются вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы ва
Generated text 4:
На судельствуются, вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы с
Generated text 5:
На судельствуются вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы во�


In [1]:
import pandas as pd
from transformers import GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments, GPT2LMHeadModel
import os

# Определение токенизатора
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")
tokenizer.pad_token = tokenizer.eos_token

def read_jokes_csv(filename):
    df = pd.read_csv(filename, usecols=['text'])
    return df

def train_and_save_model():
    # Загрузка данных
    df = read_jokes_csv("jokes.csv")
    df = df.head(20)
    train_df = df.sample(frac=0.8, random_state=42)
    eval_df = df.drop(train_df.index)

    # Сохранение данных в файлы
    train_path = "jokes_train.txt"
    test_path = "jokes_test.txt"
    train_df['text'].to_csv(train_path, index=False, header=False)
    eval_df['text'].to_csv(test_path, index=False, header=False)

    # Токенизация данных
    train_dataset = TextDataset(tokenizer=tokenizer, file_path=train_path, block_size=128, overwrite_cache=True)
    test_dataset = TextDataset(tokenizer=tokenizer, file_path=test_path, block_size=128, overwrite_cache=True)
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    # Определение параметров обучения
    training_args = TrainingArguments(
        output_dir="./jokes_model",
        overwrite_output_dir=True,
        num_train_epochs=2,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        logging_dir="./logs",
        logging_steps=10,
        save_steps=10,
        eval_steps=10,
        save_total_limit=2,
        evaluation_strategy="steps"
    )

    # Обучение модели
    model = GPT2LMHeadModel.from_pretrained("gpt2-medium")
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
    )

    # Тренировка модели
    training_results = trainer.train()
    trainer.save_model("./jokes_model")

    # Сохранение токенизатора
    tokenizer.save_pretrained("./jokes_model")

    # Возвращаем среднее значение потерь для всего обучения
    return model, tokenizer, training_results.training_loss

def load_and_test_model(model_path, test_path):
    # Загрузка модели и токенизатора из файла
    model = GPT2LMHeadModel.from_pretrained(model_path)
    tokenizer = GPT2Tokenizer.from_pretrained(model_path)
    tokenizer.pad_token = tokenizer.eos_token

    # Токенизация тестовых данных
    test_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=test_path,
        block_size=128,
        overwrite_cache=True
    )

    # Оценка модели на тестовых данных
    training_args = TrainingArguments(
        output_dir="./results_test",
        per_device_eval_batch_size=8,
        logging_dir="./logs_test",
        logging_steps=10,
        eval_steps=10,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=DataCollatorForLanguageModeling(
            tokenizer=tokenizer, mlm=False
        ),
        eval_dataset=test_dataset,
    )

    results = trainer.evaluate()

    print("Evaluation Results:", results)

    # Генерация текста на основе обученной модели
    input_text = "На суде"
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    # output = model.generate(input_ids, max_length=100, num_return_sequences=5, temperature=0.7)
    output = model.generate(input_ids, max_length=100, num_return_sequences=5, num_beams=5, temperature=0.7)
    for i, generated_text in enumerate(tokenizer.decode(o, skip_special_tokens=True) for o in output):
        print(f"Generated text {i + 1}:")
        print(generated_text)



if __name__ == "__main__":
    # Обучение и сохранение модели
    model, tokenizer, average_loss = train_and_save_model()

    # Вывод числового значения среднего loss на обучении
    print(f"Avg Training Loss: {average_loss}")

    # Загрузка модели и оценка на тестовых данных
    load_and_test_model("./jokes_model", "jokes_test.txt")

D:\data\innopolis\itog\venv\Lib\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss,Validation Loss


Avg Training Loss: 2.429222742716471


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
D:\data\innopolis\itog\venv\Lib\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


D:\data\innopolis\itog\venv\Lib\site-packages\transformers\generation\configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Evaluation Results: {'eval_loss': 1.902648687362671, 'eval_runtime': 2.3857, 'eval_samples_per_second': 2.096, 'eval_steps_per_second': 0.419}
Generated text 1:
На судельствуются вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы
Generated text 2:
На судельствуются, вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы в
Generated text 3:
На судельствуются вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы ва
Generated text 4:
На судельствуются, вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы с
Generated text 5:
На судельствуются вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы вы во�


In [2]:
import pandas as pd
from transformers import GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments, GPT2LMHeadModel
import os

# Определение токенизатора
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")
tokenizer.pad_token = tokenizer.eos_token

def read_jokes_csv(filename):
    df = pd.read_csv(filename, usecols=['text'])
    return df

def train_and_save_model():
    # Загрузка данных
    df = read_jokes_csv("jokes.csv")
    df = df.head(100)
    train_df = df.sample(frac=0.8, random_state=42)
    eval_df = df.drop(train_df.index)

    # Сохранение данных в файлы
    train_path = "jokes_train.txt"
    test_path = "jokes_test.txt"
    train_df['text'].to_csv(train_path, index=False, header=False)
    eval_df['text'].to_csv(test_path, index=False, header=False)

    # Токенизация данных
    train_dataset = TextDataset(tokenizer=tokenizer, file_path=train_path, block_size=128, overwrite_cache=True)
    test_dataset = TextDataset(tokenizer=tokenizer, file_path=test_path, block_size=128, overwrite_cache=True)
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    # Определение параметров обучения
    training_args = TrainingArguments(
        output_dir="./jokes_model",
        overwrite_output_dir=True,
        num_train_epochs=4,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        logging_dir="./logs",
        logging_steps=10,
        save_steps=10,
        eval_steps=10,
        save_total_limit=2,
        evaluation_strategy="steps"
    )

    # Обучение модели
    model = GPT2LMHeadModel.from_pretrained("gpt2-medium")
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
    )

    # Тренировка модели
    training_results = trainer.train()
    trainer.save_model("./jokes_model")

    # Сохранение токенизатора
    tokenizer.save_pretrained("./jokes_model")

    # Возвращаем среднее значение потерь для всего обучения
    return model, tokenizer, training_results.training_loss

In [3]:
def load_and_test_model(model_path, test_path):
    # Загрузка модели и токенизатора из файла
    model = GPT2LMHeadModel.from_pretrained(model_path)
    tokenizer = GPT2Tokenizer.from_pretrained(model_path)
    tokenizer.pad_token = tokenizer.eos_token

    # Токенизация тестовых данных
    test_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=test_path,
        block_size=128,
        overwrite_cache=True
    )

    # Оценка модели на тестовых данных
    training_args = TrainingArguments(
        output_dir="./results_test",
        per_device_eval_batch_size=8,
        logging_dir="./logs_test",
        logging_steps=10,
        eval_steps=10,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=DataCollatorForLanguageModeling(
            tokenizer=tokenizer, mlm=False
        ),
        eval_dataset=test_dataset,
    )

    results = trainer.evaluate()

    print("Evaluation Results:", results)

    # Генерация текста на основе обученной модели
    input_text = "Программист"
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    # output = model.generate(input_ids, max_length=100, num_return_sequences=5, temperature=0.7)
    output = model.generate(input_ids, max_length=100, num_return_sequences=5, num_beams=5, temperature=0.7)
    for i, generated_text in enumerate(tokenizer.decode(o, skip_special_tokens=True) for o in output):
        print(f"Generated text {i + 1}:")
        print(generated_text)

In [4]:
# Обучение и сохранение модели
model, tokenizer, average_loss = train_and_save_model()

print(f"Avg Training Loss: {average_loss}")

load_and_test_model("./jokes_model", "jokes_test.txt")

D:\data\innopolis\itog\venv\Lib\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss,Validation Loss


Avg Training Loss: 2.1047313610712686


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
D:\data\innopolis\itog\venv\Lib\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


D:\data\innopolis\itog\venv\Lib\site-packages\transformers\generation\configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Evaluation Results: {'eval_loss': 2.029554605484009, 'eval_runtime': 17.3146, 'eval_samples_per_second': 2.021, 'eval_steps_per_second': 0.289}
Generated text 1:
На судент приходит приходит приходит приходит приходит приходит приходит приходит приходит
Generated text 2:
На судей приходй приходй приходй приходй приходй приходй приходй приходй �
Generated text 3:
На судей приходй приходй приходй приходй приходй приходй приходй приходй.
Generated text 4:
На судей приходй приходй приходй приходй приходй приходй приходй приходй

Generated text 5:
На судей приходй приходй приходй приходй приходй приходй приходй приходй 
